In [2]:
import sys
sys.path.append("/Users/mustafa/Documents/MyWorkspaces/testdispatchuv")


In [3]:

import pandas as pd
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from googleapiclient.http import MediaFileUpload
import pickle
from datetime import datetime
from dotenv import load_dotenv
import json
import base64
from app.db.postgres import PostgresDataSource
import logging
from app.core.utils import clean_integer, decode_base64_credentials, get_google_drive_service, upload_to_drive, verifier_champs_manquants,SCOPES


In [4]:
db = PostgresDataSource() 

In [4]:
# 

2025-05-11 20:37:13 - app.db.postgres - INFO - Connexion à la base de données établie avec succès


2025-05-11 20:37:13,492 - app.db.postgres - INFO - Connexion à la base de données établie avec succès
2025-05-11 20:37:13,609 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


Rapport généré et uploadé avec succès!
Nombre total de lignes vérifiées: 74
Nombre de lignes avec erreurs: 14


In [5]:
# V2 AVEC enregistrement dans la base de données

# Configuration de la journalisation
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('hotes_verification.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)



#from google.oauth2 import service_account

# def get_google_drive_service():
#     """Version pour Service Account"""
#     encoded_credentials = os.getenv('GOOGLE_CREDENTIALS_BASE64')
#     if not encoded_credentials:
#         raise ValueError("GOOGLE_CREDENTIALS_BASE64 manquant")
    
#     credentials_dict = decode_base64_credentials(encoded_credentials)
    
#     if 'type' not in credentials_dict or credentials_dict['type'] != 'service_account':
#         raise ValueError("Les identifiants doivent être pour un Service Account")
    
#     creds = service_account.Credentials.from_service_account_info(
#         credentials_dict,
#         scopes=SCOPES
#     )
    
#     return build('drive', 'v3', credentials=creds)

# def get_google_drive_service():
#     """Initialise et retourne le service Google Drive."""
#     creds = None
    
#     # Vérifie si le token existe déjà
#     if os.path.exists('token.pickle'):
#         with open('token.pickle', 'rb') as token:
#             creds = pickle.load(token)
    
#     # Si les credentials n'existent pas ou sont invalides, demande une nouvelle authentification
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             # Récupérer les credentials encodés en base64 depuis .env
#             encoded_credentials = os.getenv('GOOGLE_CREDENTIALS_BASE64')
#             if not encoded_credentials:
#                 raise ValueError("GOOGLE_CREDENTIALS_BASE64 non trouvé dans le fichier .env")
            
#             # Décoder les credentials
#             credentials_dict = decode_base64_credentials(encoded_credentials)
            
#             # Créer le flow avec les credentials
#             flow = InstalledAppFlow.from_client_config(
#                 credentials_dict, SCOPES)
#             creds = flow.run_local_server(port=0)
        
#         # Sauvegarde les credentials pour la prochaine utilisation
#         with open('token.pickle', 'wb') as token:
#             pickle.dump(creds, token)
    
#     return build('drive', 'v3', credentials=creds)



# def upload_to_drive(file_path, folder_id):
#     """Upload un fichier vers Google Drive dans le dossier spécifié."""
#     service = get_google_drive_service()
    
#     file_metadata = {
#         'name': os.path.basename(file_path),
#         'parents': [folder_id]
#     }
    
#     media = MediaFileUpload(file_path, resumable=True)
#     file = service.files().create(
#         body=file_metadata,
#         media_body=media,
#         fields='id'
#     ).execute()
    
#     return file.get('id')



async def main():
    """Fonction principale pour vérifier les données des hôtes et reporter les erreurs."""
    # Charger les variables d'environnement
    load_dotenv()
    
    # Récupérer l'ID du dossier depuis .env
    FOLDER_ID = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
    if not FOLDER_ID:
        logger.error("GOOGLE_DRIVE_FOLDER_ID non trouvé dans le fichier .env")
        raise ValueError("GOOGLE_DRIVE_FOLDER_ID non trouvé dans le fichier .env")
    
    try:
        logger.info("Début du processus de vérification des hôtes")
        
        # 1. Récupération des données depuis la base
        columns = ["ID", "Prenom-Nom", "Telephone", "Nombre-prs-AR", "Provenance", 
                  "Arrivee-date", "Arrivee-vol", "Arrivee-heure", "Arrivee-Lieux", 
                  "Hebergeur", "RESTAURATION", "Telephone-hebergeur", "Adresse-hebergement", 
                  "Retour-date", "Nombre-prs-Ret", "Retour-vol", "Retour-heure", 
                  "Retour-Lieux", "Destination", "Chauffeur", "evenement_annee", "evenement_jour"]
        
        query = """ 
        SELECT "ID", "Prenom-Nom", "Telephone", "Nombre-prs-AR", "Provenance",
               "Arrivee-date", "Arrivee-vol", "Arrivee-heure", "Arrivee-Lieux",
               "Hebergeur", "RESTAURATION", "Telephone-hebergeur", "Adresse-hebergement",
               "Retour-date", "Nombre-prs-Ret", "Retour-vol", "Retour-heure",
               "Retour-Lieux", "Destination", "Chauffeur", "evenement_annee", "evenement_jour"
        FROM "Hotes" WHERE evenement_annee = 2024;
        """
        
        logger.info("Exécution de la requête SQL pour récupérer les données des hôtes")
        hotes_data = await db.execute_query(query)
        df = pd.DataFrame(hotes_data, columns=columns)
        logger.info(f"{len(df)} hôtes récupérés depuis la base de données")

        # Nettoyage des champs numériques
        logger.info("Nettoyage des champs numériques")
        df['ID'], df['ID_erreur'] = zip(*df['ID'].apply(clean_integer))
        df['Nombre-prs-AR'], df['Nombre-prs-AR_erreur'] = zip(*df['Nombre-prs-AR'].apply(clean_integer))
        df['Nombre-prs-Ret'], df['Nombre-prs-Ret_erreur'] = zip(*df['Nombre-prs-Ret'].apply(clean_integer))

        # 2. Vérification des données
        logger.info("Vérification des données des hôtes")
        df_verifie = verifier_donnees_hotes(df)
        
        # 3. Mise à jour des erreurs dans la base de données
        logger.info("Mise à jour des erreurs dans la table hotes")
        await update_hotes_errors(db, df_verifie)
        
        # 4. Filtrage des lignes avec erreurs
        df_erreurs = df_verifie[
            (df_verifie['erreur_aller'] != '') | 
            (df_verifie['erreur_retour'] != '') |
            (df_verifie['ID_erreur'].notna()) |
            (df_verifie['Nombre-prs-AR_erreur'].notna()) |
            (df_verifie['Nombre-prs-Ret_erreur'].notna())
        ]
        logger.info(f"{len(df_erreurs)} hôtes avec erreurs détectés")

        if not df_erreurs.empty:
            # 5. Génération du rapport d'erreurs
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            nom_fichier = f'rapport_erreurs_{timestamp}.xlsx'
            logger.info(f"Génération du rapport d'erreurs: {nom_fichier}")
            
            # Ajout des erreurs numériques dans le rapport
            df_erreurs['ID_erreur'] = df_erreurs['ID_erreur'].fillna('')
            df_erreurs['Nombre-prs-AR_erreur'] = df_erreurs['Nombre-prs-AR_erreur'].fillna('')
            df_erreurs['Nombre-prs-Ret_erreur'] = df_erreurs['Nombre-prs-Ret_erreur'].fillna('')
            
            df_erreurs.to_excel(nom_fichier, index=False)
            
            # 6. Upload vers Google Drive
            logger.info(f"Upload du rapport vers Google Drive (Dossier ID: {FOLDER_ID})")
            file_id = upload_to_drive(nom_fichier, FOLDER_ID)
            logger.info(f"Rapport uploadé avec succès (File ID: {file_id})")
            
            # 7. Nettoyage du fichier local
            #os.remove(nom_fichier)
            #logger.info("Fichier local nettoyé")
        else:
            logger.info("Aucune erreur détectée - aucun rapport généré")

        logger.info("Processus terminé avec succès")
        print(f"Nombre total de lignes vérifiées: {len(df)}")
        print(f"Nombre de lignes avec erreurs: {len(df_erreurs)}")

    except Exception as e:
        logger.error(f"Une erreur est survenue: {str(e)}", exc_info=True)
        raise

async def update_hotes_errors(db, df_verifie, batch_size=100):
    """Met à jour les champs d'erreur dans la table hotes par lots"""
    logger.info(f"Début de la mise à jour des erreurs pour {len(df_verifie)} hôtes")
    
    total_rows = len(df_verifie)
    processed = 0
    
    while processed < total_rows:
        batch = df_verifie.iloc[processed:processed+batch_size]
        transactions = []
        
        for _, row in batch.iterrows():
            # Combiner les erreurs standards et les erreurs numériques
            erreur_aller = row['erreur_aller']
            if row['ID_erreur']:
                erreur_aller += f", ID: {row['ID_erreur']}" if erreur_aller else f"ID: {row['ID_erreur']}"
            if row['Nombre-prs-AR_erreur']:
                erreur_aller += f", Nombre-prs-AR: {row['Nombre-prs-AR_erreur']}" if erreur_aller else f"Nombre-prs-AR: {row['Nombre-prs-AR_erreur']}"
            
            erreur_retour = row['erreur_retour']
            if row['Nombre-prs-Ret_erreur']:
                erreur_retour += f", Nombre-prs-Ret: {row['Nombre-prs-Ret_erreur']}" if erreur_retour else f"Nombre-prs-Ret: {row['Nombre-prs-Ret_erreur']}"
            
            update_query = """
                UPDATE "Hotes" SET
                    erreur_aller = NULLIF(%s, ''),
                    erreur_retour = NULLIF(%s, '')
                WHERE "ID" = %s
            """
            params = [
                erreur_aller,
                erreur_retour,
                row['ID']
            ]
            transactions.append((update_query, params))
        
        try:
            await db.execute_transaction(transactions)
            processed += len(batch)
            logger.info(f"Lot {processed}/{total_rows} traité avec succès")
        except Exception as e:
            logger.error(f"Erreur lors du traitement du lot {processed}-{processed+batch_size}", exc_info=True)
            raise
    
    logger.info("Mise à jour des erreurs terminée avec succès")

def verifier_donnees_hotes(df):
    """Vérifie les données des hôtes et ajoute les colonnes d'erreurs."""
    logger.info("Vérification des champs obligatoires")
    
    df_verifie = df.copy()
    
    # Champs obligatoires pour l'aller
    champs_aller = ['ID', 'Prenom-Nom', 'Nombre-prs-AR', 'Arrivee-Lieux', 
                   'Adresse-hebergement', 'Arrivee-date', 'Arrivee-heure']
    
    # Champs obligatoires pour le retour
    champs_retour = ['ID', 'Prenom-Nom', 'Nombre-prs-Ret', 'Retour-Lieux', 
                    'Retour-date', 'Retour-heure']
    
    # Vérification des champs
    df_verifie['erreur_aller'] = df_verifie.apply(
        lambda row: verifier_champs_manquants(row, champs_aller), axis=1)
    df_verifie['erreur_retour'] = df_verifie.apply(
        lambda row: verifier_champs_manquants(row, champs_retour), axis=1)
    
    return df_verifie



# Exécution dans Jupyter Notebook
await main()

2025-05-12 02:17:13,099 - __main__ - INFO - Début du processus de vérification des hôtes
2025-05-12 02:17:13,101 - __main__ - INFO - Exécution de la requête SQL pour récupérer les données des hôtes


2025-05-12 02:17:13 - app.db.postgres - INFO - Connexion à la base de données établie avec succès


2025-05-12 02:17:13,321 - app.db.postgres - INFO - Connexion à la base de données établie avec succès
2025-05-12 02:17:13,351 - __main__ - INFO - 74 hôtes récupérés depuis la base de données
2025-05-12 02:17:13,354 - __main__ - INFO - Nettoyage des champs numériques
2025-05-12 02:17:13,366 - __main__ - INFO - Vérification des données des hôtes
2025-05-12 02:17:13,376 - __main__ - INFO - Vérification des champs obligatoires
2025-05-12 02:17:13,402 - __main__ - INFO - Mise à jour des erreurs dans la table hotes
2025-05-12 02:17:13,403 - __main__ - INFO - Début de la mise à jour des erreurs pour 74 hôtes


2025-05-12 02:17:13 - app.db.postgres - INFO - Transaction démarrée


2025-05-12 02:17:13,415 - app.db.postgres - INFO - Transaction démarrée


2025-05-12 02:17:14 - app.db.postgres - INFO - Transaction validée avec succès


2025-05-12 02:17:14,088 - app.db.postgres - INFO - Transaction validée avec succès
2025-05-12 02:17:14,089 - __main__ - INFO - Lot 74/74 traité avec succès
2025-05-12 02:17:14,090 - __main__ - INFO - Mise à jour des erreurs terminée avec succès
2025-05-12 02:17:14,094 - __main__ - INFO - 15 hôtes avec erreurs détectés
2025-05-12 02:17:14,094 - __main__ - INFO - Génération du rapport d'erreurs: rapport_erreurs_20250512_021714.xlsx
/var/folders/8v/kz5040m5025c51v8c045qhcr0000gn/T/ipykernel_4119/3334442658.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erreurs['ID_erreur'] = df_erreurs['ID_erreur'].fillna('')
/var/folders/8v/kz5040m5025c51v8c045qhcr0000gn/T/ipykernel_4119/3334442658.py:158: SettingWithCopyWarning: 
A value is trying to be 

Nombre total de lignes vérifiées: 74
Nombre de lignes avec erreurs: 15


In [ ]:
# V1 SANS enregistrement dans la base de données


# Charger les variables d'environnement
# load_dotenv()

# # Configuration des scopes pour Google Drive API
# SCOPES = ['https://www.googleapis.com/auth/drive.file']

# def decode_base64_credentials(encoded_credentials):
#     """Décode les credentials encodés en base64."""
#     try:
#         # Décoder la chaîne base64
#         decoded_bytes = base64.b64decode(encoded_credentials)
#         # Convertir en chaîne de caractères
#         decoded_str = decoded_bytes.decode('utf-8')
#         # Parser le JSON
#         return json.loads(decoded_str)
#     except Exception as e:
#         raise ValueError(f"Erreur lors du décodage des credentials: {str(e)}")




# from google.oauth2 import service_account

# def get_google_drive_service():
#     """Version pour Service Account"""
#     encoded_credentials = os.getenv('GOOGLE_CREDENTIALS_BASE64')
#     if not encoded_credentials:
#         raise ValueError("GOOGLE_CREDENTIALS_BASE64 manquant")
    
#     credentials_dict = decode_base64_credentials(encoded_credentials)
    
#     if 'type' not in credentials_dict or credentials_dict['type'] != 'service_account':
#         raise ValueError("Les identifiants doivent être pour un Service Account")
    
#     creds = service_account.Credentials.from_service_account_info(
#         credentials_dict,
#         scopes=SCOPES
#     )
    
#     return build('drive', 'v3', credentials=creds)
# # def get_google_drive_service():
# #     """Initialise et retourne le service Google Drive."""
# #     creds = None
    
# #     # Vérifie si le token existe déjà
# #     if os.path.exists('token.pickle'):
# #         with open('token.pickle', 'rb') as token:
# #             creds = pickle.load(token)
    
# #     # Si les credentials n'existent pas ou sont invalides, demande une nouvelle authentification
# #     if not creds or not creds.valid:
# #         if creds and creds.expired and creds.refresh_token:
# #             creds.refresh(Request())
# #         else:
# #             # Récupérer les credentials encodés en base64 depuis .env
# #             encoded_credentials = os.getenv('GOOGLE_CREDENTIALS_BASE64')
# #             if not encoded_credentials:
# #                 raise ValueError("GOOGLE_CREDENTIALS_BASE64 non trouvé dans le fichier .env")
            
# #             # Décoder les credentials
# #             credentials_dict = decode_base64_credentials(encoded_credentials)
            
# #             # Créer le flow avec les credentials
# #             flow = InstalledAppFlow.from_client_config(
# #                 credentials_dict, SCOPES)
# #             creds = flow.run_local_server(port=0)
        
# #         # Sauvegarde les credentials pour la prochaine utilisation
# #         with open('token.pickle', 'wb') as token:
# #             pickle.dump(creds, token)
    
# #     return build('drive', 'v3', credentials=creds)



# def upload_to_drive(file_path, folder_id):
#     """Upload un fichier vers Google Drive dans le dossier spécifié."""
#     service = get_google_drive_service()
    
#     file_metadata = {
#         'name': os.path.basename(file_path),
#         'parents': [folder_id]
#     }
    
#     media = MediaFileUpload(file_path, resumable=True)
#     file = service.files().create(
#         body=file_metadata,
#         media_body=media,
#         fields='id'
#     ).execute()
    
#     return file.get('id')

# def verifier_champs_manquants(row, champs_a_verifier):
#     """Vérifie les champs manquants pour une ligne donnée."""
#     champs_manquants = []
#     for champ in champs_a_verifier:
#         if pd.isna(row[champ]) or str(row[champ]).strip() == '':
#             champs_manquants.append(champ)
#     return ', '.join(champs_manquants) if champs_manquants else ''

# def verifier_donnees_hotes(df):
#     """Vérifie les données des hôtes et ajoute les colonnes d'erreurs."""
#     # Copie du DataFrame pour éviter de modifier l'original
#     df_verifie = df.copy()
    
#     # Définition des champs à vérifier pour l'aller et le retour
#     champs_aller = ['ID', 'Prenom-Nom', 'Nombre-prs-AR', 'Arrivee-Lieux', 
#                     'Adresse-hebergement', 'Arrivee-date', 'Arrivee-heure']
#     champs_retour = ['ID', 'Prenom-Nom', 'Nombre-prs-Ret', 'Retour-Lieux', 
#                      'Retour-date', 'Retour-heure']
    
#     # Ajout des colonnes d'erreurs
#     df_verifie['erreur_aller'] = df_verifie.apply(
#         lambda row: verifier_champs_manquants(row, champs_aller), axis=1)
#     df_verifie['erreur_retour'] = df_verifie.apply(
#         lambda row: verifier_champs_manquants(row, champs_retour), axis=1)
    
#     return df_verifie

# async def main():
#     # Récupérer l'ID du dossier depuis .env
#     FOLDER_ID = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
#     if not FOLDER_ID:
#         raise ValueError("GOOGLE_DRIVE_FOLDER_ID non trouvé dans le fichier .env")
    
#     try:
#         # Lecture du fichier Excel des hôtes
#         #df = pd.read_excel('hotes.xlsx')
#         # Récupérer les données de la table Hotes avec execute_query
#         columns = ["ID", "Prenom-Nom", "Telephone", "Nombre-prs-AR", "Provenance", "Arrivee-date", "Arrivee-vol", "Arrivee-heure", "Arrivee-Lieux", "Hebergeur", "RESTAURATION", "Telephone-hebergeur", "Adresse-hebergement", "Retour-date", "Nombre-prs-Ret", "Retour-vol", "Retour-heure", "Retour-Lieux", "Destination", "Chauffeur", "evenement_annee", "evenement_jour"]
#         query = """ 
#         SELECT  "ID" ,
#             "Prenom-Nom" ,
#             "Telephone" ,
#             "Nombre-prs-AR" ,
#             "Provenance" ,
#             "Arrivee-date" ,
#             "Arrivee-vol" ,
#             "Arrivee-heure" ,
#             "Arrivee-Lieux" ,
#             "Hebergeur" ,
#             "RESTAURATION" ,
#             "Telephone-hebergeur" ,
#             "Adresse-hebergement" ,
#             "Retour-date" ,
#             "Nombre-prs-Ret" ,
#             "Retour-vol" ,
#             "Retour-heure" ,
#             "Retour-Lieux" ,
#             "Destination" ,
#             "Chauffeur" ,
#             "evenement_annee" ,
#             "evenement_jour"
#             FROM "Hotes" where evenement_annee = 2024
#             ;
#             """
        
#         hotes_data = await db.execute_query(query)

#         # Convertir les données en DataFrame
#         df = pd.DataFrame(hotes_data, columns=columns)
#         # Vérification des données
#         df_verifie = verifier_donnees_hotes(df)
        
#         # Filtrage des lignes avec erreurs
#         df_erreurs = df_verifie[
#             (df_verifie['erreur_aller'] != '') | 
#             (df_verifie['erreur_retour'] != '')
#         ]
        
#         # Génération du nom de fichier avec timestamp
#         timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#         nom_fichier = f'rapport_erreurs_{timestamp}.xlsx'
        
#         # Sauvegarde du fichier Excel
#         df_erreurs.to_excel(nom_fichier, index=False)
        
#         # Upload vers Google Drive
#         file_id = upload_to_drive(nom_fichier, FOLDER_ID)
        
#         print(f"Rapport généré et uploadé avec succès!")
#         print(f"Nombre total de lignes vérifiées: {len(df)}")
#         print(f"Nombre de lignes avec erreurs: {len(df_erreurs)}")
        
#         # Nettoyage du fichier local
#         os.remove(nom_fichier)
        
#     except Exception as e:
#         print(f"Une erreur est survenue: {str(e)}")


# # Exécution dans Jupyter Notebook
# await main()  # Remplacez asyncio.run(main()) par cette ligne